In [ ]:
"""
A. Tham khảo hưỡng dẫn về GNN tại:
+ https://viblo.asia/p/gioi-thieu-ve-graph-neural-networks-gnns-yZjJYG7MVOE
+ https://docs.dgl.ai/en/0.8.x/tutorials/blitz/4_link_predict.html
+ https://arxiv.org/ftp/arxiv/papers/1812/1812.08434.pdf

B. Yêu cầu:
1. Tìm hiểu và trình bày tổng quan về GNN
2. Sử dụng GNN để dự đoán liên kết mạng xã hội mầ bạn lựa chọn
3. Đánh giá kết quả giữa GNN và các phương pháp trong LAB 04.01 / 04.02
"""

In [1]:

!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00


In [2]:

import torch
import torch.nn.functional as F
from torch_geometric.datasets import KarateClub
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges, negative_sampling
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Tải dataset Karate Club
dataset = KarateClub()
data = dataset[0]

# Tạo dữ liệu cho bài toán dự đoán liên kết
data = train_test_split_edges(data)  # Tự động tạo tập train, val, test cho các liên kết

# Tạo các liên kết âm cho tập huấn luyện
data.train_neg_edge_index = negative_sampling(
    edge_index=data.train_pos_edge_index,
    num_nodes=data.num_nodes
)

# Xây dựng mô hình GCN
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)  # Lớp GCN đầu tiên
        self.conv2 = GCNConv(16, out_channels)  # Lớp GCN thứ hai

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)  # Tích chập đầu tiên
        x = F.relu(x)                  # Hàm kích hoạt ReLU
        x = self.conv2(x, edge_index)  # Tích chập thứ hai
        return x

# Hàm dự đoán liên kết (dot product giữa các nút)
def link_prediction(z, edge_label_index):
    return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=1)

# Khởi tạo mô hình và tham số
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(data.num_features, 16).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Huấn luyện mô hình
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    z = model(data.x, data.train_pos_edge_index)  # Biểu diễn nút học được
    pos_pred = link_prediction(z, data.train_pos_edge_index)  # Dự đoán liên kết dương
    neg_pred = link_prediction(z, data.train_neg_edge_index)  # Dự đoán liên kết âm

    # Tính loss
    loss = F.binary_cross_entropy_with_logits(
        torch.cat([pos_pred, neg_pred]),
        torch.cat([torch.ones(pos_pred.size(0)), torch.zeros(neg_pred.size(0))]).to(device)
    )
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Đánh giá mô hình
model.eval()
with torch.no_grad():
    z = model(data.x, data.train_pos_edge_index)
    pos_pred = torch.sigmoid(link_prediction(z, data.test_pos_edge_index))
    neg_pred = torch.sigmoid(link_prediction(z, data.test_neg_edge_index))

    # Tính các chỉ số đánh giá
    y_true = torch.cat([torch.ones(pos_pred.size(0)), torch.zeros(neg_pred.size(0))]).cpu().numpy()
    y_pred = torch.cat([pos_pred, neg_pred]).cpu().numpy()

    # Tính độ chính xác
    acc = accuracy_score(y_true, (y_pred > 0.5).astype(int))

    # Tính độ chính xác, recall, F1-score
    precision = precision_score(y_true, (y_pred > 0.5).astype(int))
    recall = recall_score(y_true, (y_pred > 0.5).astype(int))
    f1 = f1_score(y_true, (y_pred > 0.5).astype(int))

    # Tính AUC-ROC
    auc = roc_auc_score(y_true, y_pred)

    print(f"Độ chính xác: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Epoch 0, Loss: 0.6862
Epoch 20, Loss: 0.4825
Epoch 40, Loss: 0.3846
Epoch 60, Loss: 0.3499
Epoch 80, Loss: 0.3169
Epoch 100, Loss: 0.2630
Epoch 120, Loss: 0.2209
Epoch 140, Loss: 0.1921
Epoch 160, Loss: 0.1649
Epoch 180, Loss: 0.1374
Độ chính xác: 0.5714
Precision: 0.6667
Recall: 0.2857
F1-score: 0.4000
AUC-ROC: 0.5102


Đánh giá mô hình GNN:
Accuracy:

GNN đạt 0.5714, chỉ cao hơn Preferential Attachment (0.52) nhưng thấp hơn các mô hình khác như Common Neighbors (0.85) và Random Forest (0.9333).
Precision:

Precision của GNN là 0.6667, cao hơn Preferential Attachment (0.51087) nhưng vẫn thấp hơn Random Forest (1.0000) và các phương pháp khác.
Recall:

Recall rất thấp (0.2857) so với tất cả các mô hình còn lại, đặc biệt các mô hình truyền thống đều đạt 0.94.
F1-score:

Với 0.4000, F1-score của GNN phản ánh sự mất cân bằng nghiêm trọng giữa Precision và Recall.
AUC-ROC:

GNN đạt 0.5102, gần như ngẫu nhiên (0.5), cho thấy khả năng phân biệt giữa các lớp kém.

So sánh tổng thể:
GNN thể hiện hiệu suất kém hơn so với các mô hình truyền thống và Random Forest.
Random Forest vẫn là mô hình vượt trội với tất cả các chỉ số: AUC-ROC (0.9864), Precision (1.0000) và F1-score (0.9375).
Adamic-Adar và Common Neighbors vẫn duy trì kết quả ổn định với AUC-ROC trên 0.93.